In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pk
#import networkx as nx
import re 

path = "d:/Facultad/Tesis/"
path_git = "d:/Git_Proyects/Tesis/Base_Twitter/"
base_nac = pd.read_csv(path+'Corpus_medios_nac.csv')
Tweets_mayo = pd.read_csv(path_git+'Tweets_march_solo_medios.csv')

C:\Users\Ale\AppData\Local\Temp\ipykernel_9848\2241067710.py:10: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  base_nac = pd.read_csv(path+'Corpus_medios_nac.csv')


In [33]:
def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    
    return x

def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def words_length(x):
    return len(x.split(" "))

def pre_process_quoted(df1):
    df = df1.copy()
#    df['nota'] = df['nota'].apply(replace)
    df.loc[:, 'nota'] = df['nota'].apply(replace)
    #df['citas']  = df['nota'].apply(extract_quotes)
    df.loc[:, 'citas'] = df['nota'].apply(extract_quotes)

    df_quoutes2 = pd.DataFrame({'Fecha': df['fecha'].iloc[1:], 'Hora': df['hora'].iloc[1:], 'Cita': df['citas'].iloc[1:], 'Link': df['link'].iloc[1:], 'Nota': df['nota'].iloc[1:]})
    df_quoutes = df_quoutes2.explode('Cita') # Cada cita sea una fila distinta
    df_quoutes = df_quoutes.dropna(subset=['Cita']) # Elimino la filas que no haya quotes

    #df_quoutes['Citas'] = df_quoutes['Citas'].apply(replace)

    df_quoutes = df_quoutes.reset_index()
    df_quoutes = df_quoutes.drop(['index'], axis = 1)
    df_quoutes['Cant_Palabras'] = df_quoutes['Cita'].apply(words_length) # genero columna que cuente la cantidad de plabaras
    df_quoutes_pf = df_quoutes[df_quoutes['Cant_Palabras'] > 4]
    df_quoutes_pf = df_quoutes_pf[df_quoutes_pf['Cant_Palabras'] < 60]
    df_quoutes_pf = df_quoutes_pf.reset_index()
    df_quoutes_pf = df_quoutes_pf.drop(['index'], axis = 1)
    return df_quoutes_pf

In [34]:
base_nac['medios_comp'] = base_nac['link'].isin(Tweets_mayo['urls'])
base_compartida = base_nac[base_nac['medios_comp']==True]

In [35]:
extract_quotes  = pre_process_quoted(base_compartida)
extract_quotes

,Fecha,Hora,Cita,Link,Nota,Cant_Palabras
0,2019/01/26,NaN,No queremos que sea lo principal. Ya se discut...,https://www.clarin.com/politica/gobierno-mante...,Con el objetivo de no fogonear el debate sobre...,24
1,2019/01/26,NaN,defensa de las dos vidas,https://www.clarin.com/politica/gobierno-mante...,Con el objetivo de no fogonear el debate sobre...,5
2,2019/01/26,NaN,el tema central es el derecho de las mujeres a...,https://www.clarin.com/politica/gobierno-mante...,Con el objetivo de no fogonear el debate sobre...,11
3,2019/01/26,NaN,teniendo en cuenta los motivos que impulsaron ...,https://www.clarin.com/politica/gobierno-mante...,Con el objetivo de no fogonear el debate sobre...,36
4,2019/01/26,NaN,Estaba muy sensible el tema y la decisión es ...,https://www.clarin.com/politica/gobierno-mante...,Con el objetivo de no fogonear el debate sobre...,37
...,...,...,...,...,...,...
6869,2019/03/31,NaN,Es el único país en el mundo con tantos proc...,http://www.losprimeros.tv/nota/91885/barrionue...,El secretario general del gremio de gastronóm...,19
6870,2019/03/31,NaN,"Tuvieron una charla muy buena, cálida y larga...",http://www.losprimeros.tv/nota/91865/macri-vis...,Mauricio Macri viajó en la noche del viernes ...,11
6871,2019/03/31,NaN,Contó cómo ven al país en el mundo y cómo ...,http://www.losprimeros.tv/nota/91865/macri-vis...,Mauricio Macri viajó en la noche del viernes ...,30
6872,2019/03/31,NaN,El único candidato a presidente de Cambiemos ...,http://www.losprimeros.tv/nota/91865/macri-vis...,Mauricio Macri viajó en la noche del viernes ...,9


In [ ]:
citas = extract_quotes['Cita']
for i in citas:
    print(i)

In [39]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')  
phrases_emb = model.encode(citas, convert_to_tensor=True) 

In [57]:
clusters = util.community_detection(phrases_emb,threshold=0.8, min_community_size = 1)  
cluste_map = {citas[j]: str(i+1) for i, clus in enumerate(clusters) for j in clus}
extract_quotes['id_cluster'] = extract_quotes['Cita'].map(cluste_map)

In [63]:
for cit in extract_quotes[extract_quotes['id_cluster'] == '6']['Cita']:
    print(cit)

Macri y Cristina son dos caras de la misma moneda y tienen el boleto picado
(Mauricio) Macri y Cristina (Kirchner) son dos caras de la misma moneda. Los dos tienen picado el boleto
Macri y Cristina son dos caras de la misma moneda, tienen picado el boleto
Macri y Cristina son dos caras de la misma moneda, tienen picado el boleto
(Mauricio) Macri y Cristina (Kirchner) son dos caras de la misma moneda. Los dos tienen picado el boleto
Estamos ante dos personas, Macri y Cristina, que son dos caras de la misma moneda. Y los dos tienen picado el boleto
(Mauricio) Macri y Cristina (Kirchner) son dos caras de la misma moneda. Los dos tienen picado el boleto
Estamos ante dos personas, Macri y Cristina, que son dos caras de la misma moneda. Los dos tiene picado el boleto
Estamos ante dos personas, Macri y Cristina, que son dos caras de la misma moneda. Y los dos tienen picado el boleto
(Mauricio) Macri y Cristina (Kirchner) son dos caras de la misma moneda. Los dos tienen picado el boleto


In [ ]:
pk.dump(clusters,open(path+'oct_Tw/'+nombre+'clusters_75_oct.pk', 'wb'))

TOKEN = "6287446315:AAFAnvbB6vUSzttp-smI5E00jDP7hNI7kCo"
chat_id = "6045013691"
message = f"Termine de correr el codigo que clusteriza con 0.85 con medios nacionales con data del 07al 12  del 2021"
url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"                               
print(requests.get(url).json())